In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from scipy.io import netcdf

import iris
import iris.coord_categorisation
import iris.plot as iplt
import iris.quickplot as qplt

import os
import logging

import numpy as np

import pickle
from pathlib import Path
from cartopy.util import add_cyclic_point
logger = logging.getLogger(__name__)

In [ ]:
def get_scenarios():
    """
        Return a List containing SCENARIO tag names
    """
    MODELS = [ 'clm', 'jules' ]
    LU_SCENARIOS = [ 'nattobio','nattoaff','agtonat','agtobio','agtoaff' ]
    CLIM_SCENARIOS = [ 'ssp126', 'ssp370' ]
    CLIM_MODELS = [ 'mpi-esm1-2-hr','ukesm1-0-ll','ipsl-cm6a-lr' ]
    return CLIM_SCENARIOS, CLIM_MODELS

In [ ]:
def add_metadata(cube, field, fname):
    
    esm_id = fname.split('_')[1]
    ssp_id = fname.split('_')[2]

    iris.std_names.STD_NAMES['experiment_id'] = {'canonical units':'1'}
    
    #if not cube.coords("experiment_id"):
    climate_coord = iris.coords.AuxCoord(ssp_id, "experiment_id", units="1", long_name="experiment_id")
    cube.add_aux_coord(climate_coord)

    #if not cube.coords("realization"):
    esm_coord = iris.coords.AuxCoord(esm_id, "realization", units="1")
    cube.add_aux_coord(esm_coord)

In [ ]:
def _get_land_fraction(cube):
    """Extract land fraction as :mod:`dask.array`."""
    fx_cube = None
    land_fraction = None
    errors = []
    
    try:
        fx_cube = cube.ancillary_variable('land_area_fraction')
    except iris.exceptions.AncillaryVariableNotFoundError:
        try:
            fx_cube = cube.ancillary_variable('sea_area_fraction')
        except iris.exceptions.AncillaryVariableNotFoundError:
            errors.append('Ancillary variables land/sea area fraction not '
                          'found in cube. Check ancillary data availability.')
            return (land_fraction, errors)

    land_fraction = fx_cube.core_data()
    
    if fx_cube.var_name == 'sftlf':
        land_fraction = fx_cube.core_data() / 100.0
    if fx_cube.var_name == 'sftof':
        land_fraction = 1.0 - fx_cube.core_data() / 100.0
    
    return (land_fraction, errors)

In [ ]:
def weighting_landsea_fraction(cube, area_type):
    """Weight fields using land or sea fraction.

    This preprocessor function weights a field with its corresponding land or
    sea area fraction (value between 0 and 1). The application of this is
    important for most carbon cycle variables (and other land-surface outputs),
    which are e.g. reported in units of `kgC m-2`. This actually refers to 'per
    square meter of land/sea' and NOT 'per square meter of gridbox'. So in
    order to integrate these globally or regionally one has to both area-weight
    the quantity but also weight by the land/sea fraction.

    Parameters
    ----------
    cube : iris.cube.Cube
        Data cube to be weighted. It should have an
        :class:`iris.coords.AncillaryVariable` with standard name
        ``'land_area_fraction'`` or ``'sea_area_fraction'``. If both are
        present, only the ``'land_area_fraction'`` will be used.
    area_type : str
        Use land (``'land'``) or sea (``'sea'``) fraction for weighting.

    Returns
    -------
    iris.cube.Cube
        Land/sea fraction weighted cube.

    Raises
    ------
    TypeError
        ``area_type`` is not ``'land'`` or ``'sea'``.
    ValueError
        Land/sea fraction variables ``sftlf`` or ``sftof`` not found.
    """
    if area_type not in ('land', 'sea'):
        raise TypeError(
            f"Expected 'land' or 'sea' for area_type, got '{area_type}'")
    (land_fraction, errors) = _get_land_fraction(cube)
    if land_fraction is None:
        raise ValueError(
            f"Weighting of '{cube.var_name}' with '{area_type}' fraction "
            f"failed because of the following errors: {' '.join(errors)}")

    core_data = cube.core_data()
    if area_type == 'land':
        cube.data = core_data * land_fraction
    elif area_type == 'sea':
        cube.data = core_data * (1.0 - land_fraction)
    return cube

In [ ]:
def add_ancillary_variable(cube, temp_ancil_cube):
    """Add cube as an ancillary variable in the cube containing the data.

    Parameters
    ----------
    cube: iris.cube.Cube
        Iris cube with input data.
    temp_ancil_cube: iris.cube.Cube
        Iris cube with ancillary data.

    Returns
    -------
    iris.cube.Cube
        Cube with added ancillary variables
    """
    
    cube_left_lat = cube.coord("longitude").points[0]
    cube_right_lat = cube.coord("longitude").points[-1]

    acube_left_lat = temp_ancil_cube.coord("longitude").points[0]
    acube_right_lat = temp_ancil_cube.coord("longitude").points[-1]

    if cube_left_lat != acube_left_lat:
        ancillary_cube = temp_ancil_cube.intersection(longitude=(-180,180), ignore_bounds=True )
    else:
        ancillary_cube = temp_ancil_cube
        
    ancillary_var = iris.coords.AncillaryVariable(
        ancillary_cube.core_data(),
        standard_name=ancillary_cube.standard_name,
        units=ancillary_cube.units,
        var_name=ancillary_cube.var_name,
        attributes=ancillary_cube.attributes)
    start_dim = cube.ndim - len(ancillary_var.shape)
    cube.add_ancillary_variable(ancillary_var, range(start_dim, cube.ndim))
    logger.debug('Added %s as ancillary variable in cube of %s.',
                 ancillary_cube.var_name, cube.var_name)

In [ ]:
def main():

    make_maps=False
    region=0

    var1='cVeg'
    var2='cSoil'
    var3='cTot'
    
    SCENARIO_TITLES = ['agtobio', 'agtoaff', 'nattobio', 'noluc']
    PLOT_TITLES = ['Biocrops on ag land', 'Natural regrowth on ag land', 
                   'Afforestation on ag land', 'Biocrops on natural land',
                   'Afforestation on natural land'] 

    ssp, esm = get_scenarios()
    nclim = len(esm)
    years = np.arange(2015,2101)     
    
    # First get the no land use change data
    csoil_noluc,csoil_noluc_tr,csoil_noluc_te = retrieve_data_generic(var2,'noluc','jsbach')
    cveg_noluc,cveg_noluc_tr,cveg_noluc_te = retrieve_data_generic(var1,'noluc','jsbach')
    if var3 != 'cTot':
        var3_noluc,var3_noluc_tr,var3_noluc_te = retrieve_data_generic(var3,'noluc','jsbach')
    
    if make_maps == True:
        map_diff(var1,ssp,esm,SCENARIO_TITLES,PLOT_TITLES)
        map_diff(var2,ssp,esm,SCENARIO_TITLES,PLOT_TITLES)
        
    #PLOT THE NO LUC SCENARIO
    scenario_title = 'noluc'
    plot_title = 'JSBACH No LUC'
    
    fig,axes=plt.subplots(3,1, figsize=[8,11])


    region_title=['Global','Tropics','NH-Temperate']
    # top left: cveg
    ax=axes[0]
    ax.set_ylabel(r'$\Delta$' +var1+' (PgC) '+region_title[region],fontsize=10)
    ax.set_title(plot_title,fontsize=12)
    ax.set_xlim([2015,2100])
    ax.grid(True) 
    
    #SSP126, SSP370
    colors = ('purple','orange')
    linestyles = ('-','--','-.')
    for i1 in range(0,2):
        for i0 in range(0,nclim): 
            if region==0:
                plot_cveg = cveg_noluc[ssp[i1]][esm[i0]]
            elif region==1:
                plot_cveg = cveg_noluc_tr[ssp[i1]][esm[i0]]
            elif region==2:
                plot_cveg = cveg_noluc_te[ssp[i1]][esm[i0]]               
            ax.plot(years, plot_cveg[0]-plot_cveg[0][0], label=ssp[i1]+' '+esm[i0],
                    c=colors[i1],ls=linestyles[i0],lw=2) 
            
    # top right: csoil
    ax=axes[1]
    ax.set_ylabel(r'$\Delta$' +var2+' (PgC) '+region_title[region],fontsize=10)
    ax.set_title(plot_title,fontsize=12)
    ax.set_xlim([2015,2100])
    ax.grid(True) 

    for i1 in range(0,2):
        for i0 in range(0,nclim):
            if region==0:
                plot_csoil = csoil_noluc[ssp[i1]][esm[i0]]
            elif region==1:
                plot_csoil = csoil_noluc_tr[ssp[i1]][esm[i0]]
            elif region==2:
                plot_csoil = csoil_noluc_te[ssp[i1]][esm[i0]]  

            ax.plot(years, plot_csoil[0]-plot_csoil[0][0], label=ssp[i1]+' '+esm[i0],
                    c=colors[i1],ls=linestyles[i0],lw=2)

    
    # bottom left: ctot
    ax=axes[2]
    ax.set_ylabel(r'$\Delta$' +var3+' (PgC) '+region_title[region],fontsize=10)
    ax.set_title(plot_title,fontsize=12)
    ax.set_xlim([2015,2100])
    ax.grid(True) 

    for i1 in range(0,2):
        for i0 in range(0,nclim):
            if var3=='cTot':
                if region==0:
                    plot_csoil = csoil_noluc[ssp[i1]][esm[i0]]
                    plot_cveg = cveg_noluc[ssp[i1]][esm[i0]]
                elif region==1:
                    plot_csoil = csoil_noluc_tr[ssp[i1]][esm[i0]]
                    plot_cveg = cveg_noluc_tr[ssp[i1]][esm[i0]]
                elif region==2:
                    plot_csoil = csoil_noluc_te[ssp[i1]][esm[i0]]
                    plot_cveg = cveg_noluc_te[ssp[i1]][esm[i0]]
                plotdata = (plot_csoil[0]-plot_csoil[0][0]) + (plot_cveg[0]-plot_cveg[0][0])
                ax.plot(years, plotdata, label=esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)
            else:
                if region==0:
                    plot_var3 = var3_noluc[ssp[i1]][esm[i0]]
                elif region==1:
                    plot_var3 = var3_noluc_tr[ssp[i1]][esm[i0]]
                elif region==2:
                    plot_var3 = var3_noluc_te[ssp[i1]][esm[i0]] 
                ax.plot(years, plot_var3[0]-plot_var3[0][0], label=ssp[i1]+' '+esm[i0],
                        c=colors[i1],ls=linestyles[i0],lw=2)
    
    handles,labels = axes[0].get_legend_handles_labels()
    fig.legend(handles,labels,ncol=2,loc='lower center',fontsize=10)    
    if var3 == 'cTot':
        plt.savefig('figures/jsbach/jsbach_cpools_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)    
    else:
        plt.savefig('figures/jsbach/jsbach_cpoolsV2_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)    

#NOW GET THE LAND USE SCENARIOS AND PLOT EITHER THE DIFFERENCES OR THE RAW VALUES

    plot_diff = True
    for i2 in range(0,5):
        scenario_title = SCENARIO_TITLES[i2]
        plot_title = PLOT_TITLES[i2]
        
        csoil,csoil_tr,csoil_te = retrieve_data_generic(var2,scenario_title,'jsbach')
        cveg,cveg_tr,cveg_te = retrieve_data_generic(var1,scenario_title,'jsbach')
        if var3 != 'cTot':
            var3_glob,var3_tr,var3_te = retrieve_data_generic(var3,'noluc','jsbach')
           
        fig,axes=plt.subplots(3,1, figsize=[8,11])


        # top left: cveg
        ax=axes[0]
        ax.set_ylabel(r'$\Delta$' +var1+' (PgC) '+region_title[region],fontsize=10)
        if plot_diff==True:
            ax.set_title('JSBACH ('+plot_title+') - (no LUC scenario)',fontsize=12)
        else:
            ax.set_title('JSBACH '+plot_title,fontsize=12)
        ax.set_xlim([2015,2100])
        ax.grid(True) 

        
        for i1 in range(0,2):
            for i0 in range(0,nclim):
                if region==0:
                    plot_cveg = cveg[ssp[i1]][esm[i0]]
                    plot_cveg_noluc = cveg_noluc[ssp[i1]][esm[i0]]
                if region==1:
                    plot_cveg = cveg_tr[ssp[i1]][esm[i0]]
                    plot_cveg_noluc = cveg_noluc_tr[ssp[i1]][esm[i0]]
                if region==2:
                    plot_cveg = cveg_te[ssp[i1]][esm[i0]]
                    plot_cveg_noluc = cveg_noluc_te[ssp[i1]][esm[i0]]

                if plot_diff==True:
                    ax.plot(years, 
                             (plot_cveg[0]-plot_cveg[0][0]) - (plot_cveg_noluc[0]-plot_cveg_noluc[0][0]), 
                             label=ssp[i1]+' '+esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)
                else:
                    ax.plot(years, (plot_cveg[0]-plot_cveg[0][0]), 
                         label=ssp[i1]+' '+esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)
                   
        
        # top right: csoil
        ax=axes[1]
        ax.set_ylabel(r'$\Delta$' +var2+' (PgC) '+region_title[region],fontsize=10)
        if plot_diff==True:
            ax.set_title('JSBACH ('+plot_title+') - (no LUC scenario)',fontsize=12)
        else:
            ax.set_title('JSBACH '+plot_title,fontsize=12)
        ax.set_xlim([2015,2100])
        ax.grid(True) 

        for i1 in range(0,2):
            for i0 in range(0,nclim):
                if region==0:
                    plot_csoil = csoil[ssp[i1]][esm[i0]]
                    plot_csoil_noluc = csoil_noluc[ssp[i1]][esm[i0]]
                elif region==1:
                    plot_csoil = csoil_tr[ssp[i1]][esm[i0]]
                    plot_csoil_noluc = csoil_noluc_tr[ssp[i1]][esm[i0]]
                elif region==2:
                    plot_csoil = csoil_te[ssp[i1]][esm[i0]]
                    plot_csoil_noluc = csoil_noluc_te[ssp[i1]][esm[i0]]
                
                if plot_diff==True:
                    ax.plot(years, 
                             (plot_csoil[0]-plot_csoil[0][0]) - (plot_csoil_noluc[0]-plot_csoil_noluc[0][0]), 
                             label=esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)
                else:
                    ax.plot(years, (plot_csoil[0]-plot_csoil[0][0]), 
                             label=esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)
                    
        # bottom left: ctot
        ax=axes[2]
        ax.set_ylabel(r'$\Delta$' +var3+' (PgC) '+region_title[region],fontsize=10)
        if plot_diff==True:
            ax.set_title('JSBACH ('+plot_title+') - (no LUC scenario)',fontsize=12)
        else:
            ax.set_title('JSBACH '+plot_title,fontsize=12)
        ax.set_xlim([2015,2100])
        ax.grid(True) 

        
        for i0 in range(0,nclim):
            for i1 in range(0,2): 
                if var3=='cTot':
                    if region==0:
                        plot_csoil = csoil[ssp[i1]][esm[i0]]
                        plot_cveg = cveg[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = csoil_noluc[ssp[i1]][esm[i0]]
                        plot_cveg_noluc = cveg_noluc[ssp[i1]][esm[i0]]
                    elif region==1:
                        plot_csoil = csoil_tr[ssp[i1]][esm[i0]]
                        plot_cveg = cveg_tr[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = csoil_noluc_tr[ssp[i1]][esm[i0]]
                        plot_cveg_noluc = cveg_noluc_tr[ssp[i1]][esm[i0]]
                    elif region==2:              
                        plot_csoil = csoil_te[ssp[i1]][esm[i0]]
                        plot_cveg = cveg_te[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = csoil_noluc_te[ssp[i1]][esm[i0]]
                        plot_cveg_noluc = cveg_noluc_te[ssp[i1]][esm[i0]]
                
                    if plot_diff==True:
                        plotdata = (plot_csoil[0]-plot_csoil[0][0]) \
                        - (plot_csoil_noluc[0]-plot_csoil_noluc[0][0]) \
                        + (plot_cveg[0]-plot_cveg[0][0]) \
                        - (plot_cveg_noluc[0]-plot_cveg_noluc[0][0])
           
                    else:
                        plotdata = (plot_csoil[0]-plot_csoil[0][0]) \
                        + (plot_cveg[0]-plot_cveg[0][0])
               
                else:   #Not plotting ctot
                    if region==0:
                        plot_csoil = var3_glob[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = var3_noluc[ssp[i1]][esm[i0]]
                    elif region==1:
                        plot_csoil = var3_tr[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = var3_noluc_tr[ssp[i1]][esm[i0]]
                    elif region==2:
                        plot_csoil = var3_te[ssp[i1]][esm[i0]]
                        plot_csoil_noluc = var3_noluc_te[ssp[i1]][esm[i0]]
                
                    if plot_diff==True:
                        plotdata = (plot_csoil[0]-plot_csoil[0][0]) - \
                                   (plot_csoil_noluc[0]-plot_csoil_noluc[0][0])
                    else:
                        plotdata = plot_csoil[0]-plot_csoil[0][0]
                             
                
                ax.plot(years, plotdata, label=esm[i0],c=colors[i1],ls=linestyles[i0],lw=2)


        handles,labels = axes[0].get_legend_handles_labels()
        fig.legend(handles,labels,ncol=2,loc='lower center',fontsize=10)    
        if var3=='cTot':
            if plot_diff==True:
                plt.savefig('figures/jsbach/jsbach_cpoolsdiff_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)
            else:
                plt.savefig('figures/jsbach/jsbach_cpools_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)
        else:
            if plot_diff==True:
                plt.savefig('figures/jsbach/jsbach_cpoolsdiffV2_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)
            else:
                plt.savefig('figures/jsbach/jsbach_cpoolsV2_'+region_title[region]+'_'+scenario_title+'.png', format="png", dpi=400)       
        #plt.show()


In [ ]:
run plot_tools.ipynb

In [ ]:
if __name__ == "__main__":
    main()